# Step 0: Import and Preview Dataset

In this step, we import the necessary libraries and load the raw `Superstore.csv` dataset using pandas. We'll also take an initial look at the data to understand its structure.

In [1]:
import pandas as pd

# Load CSV with correct encoding
file_path = '../data/Superstore.csv'
df = pd.read_csv(file_path, encoding='cp1252')  # Use 'cp1252' to avoid UnicodeDecodeError

# Preview shape and first few rows
print(f"Dataset shape: {df.shape}")
df.head()

Dataset shape: (9994, 21)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


# Step 1: Clean Column Names & Inspect Data Types

In this step, we clean the column names by converting them to lowercase and replacing spaces with underscores for easier access in Python.  
We'll also inspect the data types to identify columns that may need conversion (e.g., dates).

# In addition, why replace spaces with underscores?
Because:
- Underscores are valid in Python variable names
- They make column names compatible with dot notation
- They follow PEP 8 naming style: lower_case_with_underscores

In [2]:
# Show original column names
print("Original column names:")
print(df.columns.tolist())

# Clean column names: lowercase + replace spaces with underscores
df.columns = [col.strip().lower().replace(' ', '_') for col in df.columns]

# Confirm the changes
print("\nCleaned column names:")
print(df.columns.tolist())

# Check data types of all columns
print("\nData types:")
print(df.dtypes)


Original column names:
['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

Cleaned column names:
['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'state', 'postal_code', 'region', 'product_id', 'category', 'sub-category', 'product_name', 'sales', 'quantity', 'discount', 'profit']

Data types:
row_id             int64
order_id          object
order_date        object
ship_date         object
ship_mode         object
customer_id       object
customer_name     object
segment           object
country           object
city              object
state             object
postal_code        int64
region            object
product_id        object
category          object
sub-category      object
product_name      obj

# Step 2: Convert Date Columns

The `order_date` and `ship_date` columns are currently stored as strings.  
We’ll convert them to `datetime` objects using `pd.to_datetime()` so we can:
- Extract year, month, day
- Calculate shipping duration
- Group by month or year

We’ll also handle any parsing errors gracefully.


In [3]:
# Convert order_date and ship_date to datetime
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')

# Check for any parsing errors (NaT = Not a Time)
print("Order date conversion - null values:", df['order_date'].isnull().sum())
print("Ship date conversion - null values:", df['ship_date'].isnull().sum())

# Check data types of all columns
print("\nData types:")
print(df[['order_date', 'ship_date']].dtypes)

# Preview a few rows
df[['order_date', 'ship_date']].head()

Order date conversion - null values: 0
Ship date conversion - null values: 0

Data types:
order_date    datetime64[ns]
ship_date     datetime64[ns]
dtype: object


,order_date,ship_date
0,2016-11-08,2016-11-11
1,2016-11-08,2016-11-11
2,2016-06-12,2016-06-16
3,2015-10-11,2015-10-18
4,2015-10-11,2015-10-18


# Step 3: Check and Remove Duplicates

In this step, we check for and remove any duplicate rows in the dataset.  
Duplicate transactions can affect total sales, average profit, and other metrics, so it’s important to ensure each row represents a unique order line.


In [4]:
# Count duplicate rows
duplicate_count = df.duplicated().sum()
print(f"Duplicate rows found: {duplicate_count}")

# Remove duplicates
df = df.drop_duplicates()

# Confirm shape after removal
print(f"New dataset shape: {df.shape}")


Duplicate rows found: 0
New dataset shape: (9994, 21)


# Step 4: Check for and Handle Missing Values

We now check for any missing (`NaN`) values in the dataset.  
Handling missing data is crucial for avoiding errors during analysis and ensuring our insights are based on complete information.

We'll:
- Count missing values per column
- Decide on actions: drop or fill, based on impact


In [5]:
# Count missing values in each column
missing = df.isnull().sum()

# Show only columns with missing values
missing = missing[missing > 0]
missing


Series([], dtype: int64)

# Step 5: Export Cleaned Dataset

Now that the data is clean with no duplicates or missing values, we save this cleaned version for downstream analysis and visualization.


In [6]:
df.to_csv('../data/superstore_cleaned.csv', index=False)
print("Cleaned dataset saved successfully.")

Cleaned dataset saved successfully.
